In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM
llm = LLM(
    model="gemini/gemini-2.5-pro",
    temperature=0.1
)

In [4]:
original_email="""
Hello team,

just wanted to tell you all that TSRs need to focus on issue resolution with in stated SLA and if they are any facing challenges its adviced to involve their respective SME and if not helpful then engage your leads / TSE on top of it. Welcoming to pinging managment but any exception will not be tolareted if any process missed out from any one.

Thnx
"""

from crewai.tools import BaseTool
class ReplaceJargonTools(BaseTool):
    name: str = "Replace Jargons"
    description: str = "Replace Jargons with more specific terms"

    def _run(self, email:str) -> str:
        replacement = {
            "TSR" : "Technical Support Representative",
            "SLA" : "Service Level Agreement",
            "SME" : "Subject Matter Expert",
            "TSE" : "Technical Solutions Engineer",
            "ping" : "reach out"
        }

        suggestions = []
        email_lower = original_email.lower()
        for jargon, replacement in replacement.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}.")
        return "\n".join(suggestions) if suggestions else "No jargon found."


# test the custom tool `ReplaceJargonTools()`
jt = ReplaceJargonTools()
jt.run(original_email)

Using Tool: Replace Jargons


"Consider replacing 'TSR' with 'Technical Support Representative.\nConsider replacing 'SLA' with 'Service Level Agreement.\nConsider replacing 'SME' with 'Subject Matter Expert.\nConsider replacing 'TSE' with 'Technical Solutions Engineer.\nConsider replacing 'ping' with 'reach out."

In [8]:
from crewai import Agent, Task, Crew

email_agent = Agent(
  role="Email Assistant Agent",
  goal="Improve emails and make them sound professional and clear.",
  backstory="A highly qualified communication specialist who helps in writing professional emails.",
  tools=[jt],
  verbose=True,
  llm=llm
)

# The f-string is necessary to inject the 'original_email' variable into the description
email_task = Task(
  description=f"Take the rough email below and rewrite it into professional language. Expand all abbreviations.\n\nEmail: {original_email}",
  agent=email_agent,
  expected_output="A professional email written according to industry standards, with a clear subject, professional tone, and well-structured requests."
)

crew = Crew(
  agents=[email_agent],
  tasks=[email_task],
  verbose=True
)

result = crew.kickoff()
print("## Here is the polished email: ##")
print(result)

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 95f41ec7-5665-4393-b688-3b09c1b6add1                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯


🚀 Crew: crew
└── 📋 Task: 673a8d46-ad58-4558-ba3d-0a009c401b3c
    Status: Executing Task...
╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Email 